In [ ]:
STUDENT_ID = "<Emelianov Timofey 412>"

Датасет вы можете скачать тут https://www.kaggle.com/c/dogs-vs-cats

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
!cp -r /content/gdrive/'My Drive'/train train

In [ ]:
import pandas as pd
import pathlib
import hashlib
import numpy as np
import random


train_directory = pathlib.Path('/content/gdrive/My Drive/train')
print(train_directory)
sample_size = 5000

def initialize_random_seed():
    """Инициализирует ГПСЧ из STUDENT_ID"""
    sha256 = hashlib.sha256()
    sha256.update(STUDENT_ID.encode("utf-8"))
    
    fingerprint = int(sha256.hexdigest(), 16) % (2**32)
    random.seed(fingerprint)
    np.random.seed(fingerprint)


def read_target_variable():
    """Прочитаем разметку фотографий из названий файлов"""
    target_variable = {
        "filename": [],
        "is_cat": []
    }
    image_paths = list(train_directory.glob("*.jpg"))
    random.shuffle(image_paths)
    for image_path in image_paths[:sample_size]:
        filename = image_path.name
        class_name = filename.split(".")[0]
        target_variable["filename"].append(filename)
        target_variable["is_cat"].append(class_name == "cat")

    return pd.DataFrame(data=target_variable)


initialize_random_seed()

target_df = read_target_variable()

/content/gdrive/My Drive/train


In [ ]:
from PIL import Image
from tqdm import tqdm_notebook


def read_data(target_df):
    """Читает данные изображений и строит их признаковое описание"""
    image_size = (100, 100)
    features = []
    target = []
    for i, image_name, is_cat in tqdm_notebook(target_df.itertuples(), total=len(target_df)):
        image_path = str(train_directory / image_name)
        image = Image.open(image_path)
        image = image.resize(image_size) # уменьшаем изображения
        image = image.convert('LA') # преобразуем в Ч\Б
        pixels = np.asarray(image)[:, :, 0]
        pixels = pixels.flatten()
        features.append(pixels)
        target.append(is_cat)
    return np.array(features), np.array(target)

features, target = read_data(target_df)

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_random_state

x, x_test, y, y_test = train_test_split(features, target, test_size=0.2, train_size=0.8)
x_train, x_v, y_train, y_v = train_test_split(x, y, test_size = 0.25, train_size =0.75)

In [ ]:
scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_v = scaler.fit_transform(x_v)
x_test = scaler.transform(x_test)

In [ ]:
clf = linear_model.SGDClassifier(loss="log", learning_rate = "constant", eta0 = 0.8)
clf.fit(x_train, y_train)

score_train = clf.score(x_train, y_train)
score_v = clf.score(x_v, y_v)
score_test = clf.score(x_test, y_test)
print(score_train, score_v, score_test)

0.7193333333333334 0.628 0.587


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)

knn.fit(x_train, y_train)

score_train = knn.score(x_train, y_train)
score_test = knn.score(x_test, y_test)
print(score_train, score_test)

0.7976666666666666 0.666


In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf=RandomForestClassifier(n_estimators=220)
clf.fit(x_train, y_train)

score_test = clf.score(x_test, y_test)
print(score_test)

0.704


In [ ]:
from sklearn.model_selection import cross_val_score

print(cross_val_score(clf, x_train, y_train, cv=5))

[0.69166667 0.69333333 0.69166667 0.70333333 0.68333333]
